In [1]:
import numpy as np
import pandas as pd

In [2]:
# all teams in dataset 

teams = ['angels','As','astros','bluejays','braves','brewers','cardinals','cubs','dbacks','dodgers','giants','indians',\
        'mariners','marlins','mets','nationals','orioles','padres','phillies','pirates','rangers','rays','reds','redsox',\
        'rockies','royals','tigers','twins','whitesox','yankees']

In [3]:
frames = []
frames_1 = []
i = 0

while i < len(teams):
    csv = f'statcast_all_pitches/{teams[i]}_all_pitches.csv'
    dataframe = pd.read_csv(csv, low_memory=False)
    # removing empty columns then adding batters team
    dataframe = dataframe.drop(columns=['spin_dir', 'spin_rate_deprecated','break_angle_deprecated','break_length_deprecated',"tfs_deprecated","tfs_zulu_deprecated","umpire"])
    dataframe['bat_team'] = np.where(dataframe['inning_topbot']=='Bot', dataframe['home_team'], dataframe['away_team'])
    dataframe['pitch_team'] = np.where(dataframe['inning_topbot']=='Top', dataframe['home_team'], dataframe['away_team'])
    # filling missing data
    dataframe.iloc[:,0] = dataframe.iloc[:,0].fillna('XX')
    #dataframe.iloc[:,2] = dataframe.iloc[:,2].fillna('Unknown')
    #dataframe.iloc[:,3] = dataframe.iloc[:,3].fillna('Unknown')
    #dataframe.iloc[:,4] = dataframe.iloc[:,4].fillna("Unknown")
    #dataframe.iloc[:,8] = dataframe.iloc[:,8].fillna("no_result")
    #dataframe.iloc[:,10] = dataframe.iloc[:,10].fillna("Unknown")
    #dataframe.iloc[:,18] = dataframe.iloc[:,18].fillna("no_result")
    #dataframe.iloc[:,19] = dataframe.iloc[:,19].fillna("no_result")
    #dataframe.iloc[:,23] = dataframe.iloc[:,23].fillna("Unknown")
    #dataframe.iloc[:,24] = dataframe.iloc[:,24].fillna("Unknown")
    #dataframe.iloc[:,25] = dataframe.iloc[:,25].fillna("Unknown")
    #dataframe.iloc[:,26] = dataframe.iloc[:,26].fillna("Unknown")
    dataframe.iloc[:,27] = dataframe.iloc[:,27].fillna("Vacant")
    dataframe.iloc[:,28] = dataframe.iloc[:,28].fillna("Vacant")
    dataframe.iloc[:,29] = dataframe.iloc[:,29].fillna("Vacant")
    #dataframe.iloc[:,33] = dataframe.iloc[:,33].fillna("no_result")
    #dataframe.iloc[:,34] = dataframe.iloc[:,34].fillna("no_result")
    #dataframe.iloc[:,37] = dataframe.iloc[:,37].fillna("Unknown")
    #dataframe.iloc[:,38] = dataframe.iloc[:,38].fillna("Unknown")
    #dataframe.iloc[:,39] = dataframe.iloc[:,39].fillna("Unknown")
    #dataframe.iloc[:,40] = dataframe.iloc[:,40].fillna("Unknown")
    #dataframe.iloc[:,41] = dataframe.iloc[:,41].fillna("Unknown")
    #dataframe.iloc[:,42] = dataframe.iloc[:,42].fillna("Unknown")
    #dataframe.iloc[:,45] = dataframe.iloc[:,45].fillna("no_result")
    #dataframe.iloc[:,46] = dataframe.iloc[:,46].fillna("no_result")
    #dataframe.iloc[:,47] = dataframe.iloc[:,47].fillna("no_result")
    #dataframe.iloc[:,48] = dataframe.iloc[:,48].fillna("Unknown")
    #dataframe.iloc[:,49] = dataframe.iloc[:,49].fillna("Unknown")
    #dataframe.iloc[:,50] = dataframe.iloc[:,50].fillna("Unknown")
    #dataframe.iloc[:,61] = dataframe.iloc[:,61].fillna("Unknown")
    #dataframe.iloc[:,62:69] = dataframe.iloc[:,62:69].fillna("no_result")
    dataframe.iloc[:,71] = dataframe.iloc[:,71].fillna("XX")
    #dataframe.iloc[:,80:82] = dataframe.iloc[:,80:82].fillna("Unknown")
    frames_1.append(dataframe)
    # seperating batter's name from 'des' column then putting it together with batter_id
    batter_id = dataframe.iloc[:,6]
    at_bat_string = dataframe.iloc[:,11].str.split(expand = True)
    first_name = at_bat_string.iloc[:,0]
    last_name = at_bat_string.iloc[:,1]
    fixing_bugs = at_bat_string.iloc[:,2]
    batters_team = dataframe.loc[:,'bat_team']
    batters = [batter_id,batters_team, first_name, last_name,fixing_bugs]
    bat_df = pd.DataFrame(batters)
    bat_df = bat_df.transpose()
    bat_df = bat_df.rename(columns={"batter": "batter_id","bat_team":"team", 0: "first_name",1: "last_name",2:"fixing_bugs"})
    # catching bugs
    bat_df = bat_df[bat_df.first_name != "Umpire"]
    bat_df = bat_df[bat_df.first_name != "With"]
    bat_df = bat_df[bat_df.first_name != "Double"]
    bat_df = bat_df[bat_df.first_name != "Offensive"]
    bat_df = bat_df[bat_df.first_name != "Orioles"]
    bat_df = bat_df[bat_df.last_name != "challenge"]
    bat_df = bat_df[bat_df.last_name != "challenged"]
    bat_df = bat_df[bat_df.last_name != "play,"]
    bat_df = bat_df[bat_df.last_name != "Substitution:"]
    bat_df = bat_df[bat_df.fixing_bugs != "challenged"]
    bat_df = bat_df.drop(columns = ["fixing_bugs"])
    bat_df = bat_df.dropna(axis = 0)
    bat_df.drop_duplicates(inplace=True)
    frames.append(bat_df)
    # all bat indexes to csv
    i += 1
batter_id = pd.concat(frames)
batter_id = batter_id.drop_duplicates(subset="batter_id")
# send to csv for cleaning
batter_id.to_csv("bat_id.csv")
statcast = pd.concat(frames_1)    
# around 15 players had initials or "de" in their name. Fixed in Excel
# Checked for errors multiple times and i think i caught them all
len(batter_id)

989

In [4]:
statcast.head()
statcast['batter'].nunique()

990

In [5]:
#statcast['batter'].value_counts()

In [6]:
# if you are trying to do this on your own, this part is where I swapped out the clean bat_id for the one with names "J. "
# This line would break code if "bat_id_clean.csv" doesnt exist
clean_bats = pd.read_csv("bat_id_clean.csv", low_memory=False)
batter_id = pd.DataFrame(clean_bats).drop_duplicates()

In [7]:
# combining first and last names to get full name
batter_id['full_name'] = batter_id['first_name'] + " " + batter_id['last_name']
# data type was an object and the other dataset was int64
batter_id['batter_id'] = batter_id['batter_id'].astype(object)

batter_id['batter'] = batter_id['batter_id']
statcast['batter'] = statcast['batter'].astype(object)




In [8]:
# This checks to see if the dtypes are the same for merging
#statcast.dtypes
#batter_id.dtypes

In [9]:
#statcast['batter'].value_counts()
statcast['batter'].nunique()

990

In [10]:
#These 2 numbers should be the same
len(batter_id)

990

In [11]:
#batter_id.set_index('batter_id')
stats = statcast
stats.set_index('batter')
# combining data into new dataFrame
new_df = statcast.merge(batter_id,how='left',on='batter')

# cleaning up columns for new_df

# checking to see if everything is displayed the way i want
pd.set_option('display.max_columns', 100)

new_df.head(2)

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,fielder_2,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,bat_team,pitch_team,Unnamed: 0,batter_id,team,first_name,last_name,full_name
0,FF,2018-09-30,88.8,-1.6034,6.8141,Parker Bridwell,592192,592170,field_out,hit_into_play,5.0,"Mark Canha grounds out, third baseman Taylor W...",R,R,R,LAA,OAK,X,5.0,ground_ball,0,0,2018,-0.6262,1.5243,-0.2405,2.7571,Vacant,Vacant,Vacant,2,9,Top,98.66,163.42,516472.0,180930_214100,4.5405,-128.9477,-6.8164,-7.7813,28.8214,-14.4520,3.467,1.589,4.0,61.0,-35.0,87.242,2435.0,5.590,531832,592170,516472.0,527043.0,582473.0,621493.0,592230.0,458913.0,641684.0,518466.0,54.9098,0.058,0.044,0.0,1.0,0.0,0.0,2.0,63,1,4-Seam Fastball,2,4,4,2,4,2,4,2,Standard,Standard,OAK,LAA,0,592192,OAK,Mark,Canha,Mark Canha
1,FF,2018-09-30,91.1,-1.5852,6.7128,Parker Bridwell,620439,592170,field_out,hit_into_play,13.0,Franklin Barreto pops out to shortstop Kaleb C...,R,R,R,LAA,OAK,X,6.0,popup,2,2,2018,-0.6568,1.5896,-0.8574,2.3210,Vacant,Vacant,Vacant,1,9,Top,93.82,122.84,516472.0,180930_214031,3.1474,-132.1923,-8.2514,-8.1771,30.0487,-12.5102,3.301,1.504,199.0,93.5,61.0,89.459,2457.0,5.584,531832,592170,516472.0,527043.0,582473.0,621493.0,592230.0,458913.0,641684.0,518466.0,54.9157,0.008,0.008,0.0,1.0,0.0,0.0,3.0,62,5,4-Seam Fastball,2,4,4,2,4,2,4,2,NaN,NaN,OAK,LAA,1,620439,OAK,Franklin,Barreto,Franklin Barreto


In [12]:
# Checking to see if there are null values in batter full name
# missing value was for Jose Castillo id#:620454

len(new_df)
new_df['full_name'].isnull().values.any()


null_columns=new_df.columns[new_df.isnull().any()]
print(new_df[new_df['full_name'].isnull()][null_columns])
new_df.iloc[109200:109205,:]

Empty DataFrame
Columns: [release_speed, release_pos_x, release_pos_z, events, zone, des, hit_location, bb_type, pfx_x, pfx_z, plate_x, plate_z, hc_x, hc_y, fielder_2, sv_id, vx0, vy0, vz0, ax, ay, az, sz_top, sz_bot, hit_distance_sc, launch_speed, launch_angle, effective_speed, release_spin_rate, release_extension, fielder_2.1, fielder_3, fielder_4, fielder_5, fielder_6, fielder_7, fielder_8, fielder_9, release_pos_y, estimated_ba_using_speedangle, estimated_woba_using_speedangle, woba_value, woba_denom, babip_value, iso_value, launch_speed_angle, if_fielding_alignment, of_fielding_alignment]
Index: []


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,fielder_2,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,bat_team,pitch_team,Unnamed: 0,batter_id,team,first_name,last_name,full_name
109200,SL,2018-06-15,82.3,2.0745,6.3069,Luiz Gohara,520471,627894,NaN,swinging_strike_blocked,13.0,NaN,R,R,L,ATL,SD,S,NaN,NaN,0,0,2018,-0.1014,0.0023,-1.6534,0.7761,Vacant,592669,Vacant,2,9,Top,NaN,NaN,435559.0,180616_021920,-8.1100,-119.3802,-5.8657,0.6583,23.5041,-31.2654,3.0687,1.3117,NaN,NaN,NaN,80.726,2346.0,5.440,530446,627894,435559.0,518692.0,645277.0,622666.0,621020.0,518586.0,542255.0,455976.0,55.0578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77,1,Slider,3,8,8,3,8,3,8,3,Standard,Standard,SD,ATL,5897,520471,SD,Freddy,Galvis,Freddy Galvis
109201,FF,2018-06-15,93.0,1.7610,6.4947,Luiz Gohara,620454,627894,field_out,hit_into_play_score,5.0,"Braves challenged (force play), call on the fi...",R,L,L,ATL,SD,X,6.0,ground_ball,1,2,2018,0.6506,1.2458,-0.2657,2.9657,517369,Vacant,592669,1,9,Top,127.18,149.42,435559.0,180616_021434,-6.5332,-135.1840,-5.9153,9.2196,27.8745,-16.1062,3.8025,1.5237,5.0,94.1,-31.0,92.389,1967.0,5.861,530446,627894,435559.0,518692.0,645277.0,622666.0,621020.0,518586.0,542255.0,455976.0,54.6368,0.092,0.074,0.0,1.0,0.0,0.0,2.0,76,4,4-Seam Fastball,3,7,7,3,7,3,7,3,Standard,Standard,SD,ATL,69999,620454,SD,Jose,Castillo,Jose Castillo
109202,FF,2018-06-15,92.5,1.9794,6.4527,Luiz Gohara,620454,627894,NaN,swinging_strike,12.0,NaN,R,L,L,ATL,SD,S,NaN,NaN,1,1,2018,0.8622,0.9194,1.2087,3.1358,517369,Vacant,592669,1,9,Top,NaN,NaN,435559.0,180616_021415,-3.7613,-134.6338,-4.6047,11.0783,27.3024,-20.4699,3.8025,1.5237,NaN,NaN,NaN,91.863,1949.0,5.744,530446,627894,435559.0,518692.0,645277.0,622666.0,621020.0,518586.0,542255.0,455976.0,54.7537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76,3,4-Seam Fastball,3,7,7,3,7,3,7,3,Standard,Standard,SD,ATL,69999,620454,SD,Jose,Castillo,Jose Castillo
109203,FF,2018-06-15,92.8,1.7343,6.4967,Luiz Gohara,620454,627894,NaN,called_strike,1.0,NaN,R,L,L,ATL,SD,S,NaN,NaN,1,0,2018,0.5274,1.2260,-0.3275,3.3856,517369,Vacant,592669,1,9,Top,NaN,NaN,435559.0,180616_021401,-6.3488,-134.8986,-4.7839,7.6570,27.5100,-16.6333,3.7611,1.8762,NaN,NaN,NaN,92.248,1974.0,5.874,530446,627894,435559.0,518692.0,645277.0,622666.0,621020.0,518586.0,542255.0,455976.0,54.6234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76,2,4-Seam Fastball,3,7,7,3,7,3,7,3,Standard,Standard,SD,ATL,69999,620454,SD,Jose,Castillo,Jose Castillo
109204,FT,2018-06-15,93.5,1.8403,6.4528,Luiz Gohara,620454,627894,NaN,ball,11.0,NaN,R,L,L,ATL,SD,B,NaN,NaN,0,0,2018,0.9453,1.0254,-0.0543,4.0696,517369,Vacant,592669,1,9,Top,NaN,NaN,435559.0,180616_021342,-6.8115,-135.9943,-2.6050,12.8629,27.1978,-19.4090,3.8025,1.9384,NaN,NaN,NaN,92.633,1905.0,5.599,530446,627894,435559.0,518692.0,645277.0,622666.0,621020.0,518586.0,542255.0,455976.0,54.8981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76,1,2-Seam Fastball,3,7,7,3,7,3,7,3,Strategic,Strategic,SD,ATL,69999,620454,SD,Jose,Castillo,Jose Castillo


In [13]:

new_df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,fielder_2,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,bat_team,pitch_team,Unnamed: 0,batter_id,team,first_name,last_name,full_name
0,FF,2018-09-30,88.8,-1.6034,6.8141,Parker Bridwell,592192,592170,field_out,hit_into_play,5.0,"Mark Canha grounds out, third baseman Taylor W...",R,R,R,LAA,OAK,X,5.0,ground_ball,0,0,2018,-0.6262,1.5243,-0.2405,2.7571,Vacant,Vacant,Vacant,2,9,Top,98.66,163.42,516472.0,180930_214100,4.5405,-128.9477,-6.8164,-7.7813,28.8214,-14.4520,3.4670,1.5890,4.0,61.0,-35.0,87.242,2435.0,5.590,531832,592170,516472.0,527043.0,582473.0,621493.0,592230.0,458913.0,641684.0,518466.0,54.9098,0.058,0.044,0.0,1.0,0.0,0.0,2.0,63,1,4-Seam Fastball,2,4,4,2,4,2,4,2,Standard,Standard,OAK,LAA,0,592192,OAK,Mark,Canha,Mark Canha
1,FF,2018-09-30,91.1,-1.5852,6.7128,Parker Bridwell,620439,592170,field_out,hit_into_play,13.0,Franklin Barreto pops out to shortstop Kaleb C...,R,R,R,LAA,OAK,X,6.0,popup,2,2,2018,-0.6568,1.5896,-0.8574,2.3210,Vacant,Vacant,Vacant,1,9,Top,93.82,122.84,516472.0,180930_214031,3.1474,-132.1923,-8.2514,-8.1771,30.0487,-12.5102,3.3010,1.5040,199.0,93.5,61.0,89.459,2457.0,5.584,531832,592170,516472.0,527043.0,582473.0,621493.0,592230.0,458913.0,641684.0,518466.0,54.9157,0.008,0.008,0.0,1.0,0.0,0.0,3.0,62,5,4-Seam Fastball,2,4,4,2,4,2,4,2,NaN,NaN,OAK,LAA,1,620439,OAK,Franklin,Barreto,Franklin Barreto
2,CH,2018-09-30,84.2,-1.7385,6.7809,Parker Bridwell,620439,592170,NaN,ball,13.0,NaN,R,R,R,LAA,OAK,B,NaN,NaN,1,2,2018,-1.0069,0.8183,-0.0089,0.5169,Vacant,Vacant,Vacant,1,9,Top,NaN,NaN,516472.0,180930_213956,5.8487,-122.0896,-9.5506,-10.9319,23.1999,-22.6247,3.3780,1.5589,NaN,NaN,NaN,82.920,1450.0,5.557,531832,592170,516472.0,527043.0,582473.0,621493.0,592230.0,458913.0,641684.0,518466.0,54.9423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62,4,Changeup,2,4,4,2,4,2,4,2,Standard,Standard,OAK,LAA,1,620439,OAK,Franklin,Barreto,Franklin Barreto
3,FC,2018-09-30,89.1,-1.6598,6.6491,Parker Bridwell,620439,592170,NaN,foul,4.0,NaN,R,R,R,LAA,OAK,S,NaN,NaN,1,1,2018,-0.0440,1.2437,-0.3059,2.6636,Vacant,Vacant,Vacant,1,9,Top,NaN,NaN,516472.0,180930_213932,3.3880,-129.5434,-6.2649,-1.1985,27.1724,-17.3212,3.3010,1.5040,118.0,73.8,10.0,88.382,2501.0,5.897,531832,592170,516472.0,527043.0,582473.0,621493.0,592230.0,458913.0,641684.0,518466.0,54.6023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62,3,Cutter,2,4,4,2,4,2,4,2,Standard,Standard,OAK,LAA,1,620439,OAK,Franklin,Barreto,Franklin Barreto
4,CU,2018-09-30,79.8,-1.7831,6.6868,Parker Bridwell,620439,592170,NaN,ball,14.0,NaN,R,R,R,LAA,OAK,B,NaN,NaN,0,1,2018,0.3753,-0.0491,0.8143,-0.0911,Vacant,Vacant,Vacant,1,9,Top,NaN,NaN,516472.0,180930_213913,4.9302,-115.8090,-7.8685,2.2851,22.6237,-31.3419,3.3523,1.5848,NaN,NaN,NaN,78.074,2837.0,5.324,531832,592170,516472.0,527043.0,582473.0,621493.0,592230.0,458913.0,641684.0,518466.0,55.1756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62,2,Curveball,2,4,4,2,4,2,4,2,Standard,Standard,OAK,LAA,1,620439,OAK,Franklin,Barreto,Franklin Barreto


In [14]:
len(new_df)

721190

In [15]:
#these 2 numbers would be different if something was messed up
len(statcast)


721190

In [16]:
new_df = new_df.rename(index=str, columns={"des": "description", "full_name": "batter_name"})
new_df = new_df.drop(columns=['batter', 'Unnamed: 0','first_name','last_name','team'])
new_df.head(1)

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,pitcher,events,description,zone,description,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,fielder_2,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,bat_team,pitch_team,batter_id,batter_name
0,FF,2018-09-30,88.8,-1.6034,6.8141,Parker Bridwell,592170,field_out,hit_into_play,5.0,"Mark Canha grounds out, third baseman Taylor W...",R,R,R,LAA,OAK,X,5.0,ground_ball,0,0,2018,-0.6262,1.5243,-0.2405,2.7571,Vacant,Vacant,Vacant,2,9,Top,98.66,163.42,516472.0,180930_214100,4.5405,-128.9477,-6.8164,-7.7813,28.8214,-14.452,3.467,1.589,4.0,61.0,-35.0,87.242,2435.0,5.59,531832,592170,516472.0,527043.0,582473.0,621493.0,592230.0,458913.0,641684.0,518466.0,54.9098,0.058,0.044,0.0,1.0,0.0,0.0,2.0,63,1,4-Seam Fastball,2,4,4,2,4,2,4,2,Standard,Standard,OAK,LAA,592192,Mark Canha


In [17]:
# final step to get cleaned dataset in csv for outer use
new_df.to_csv(f'all_clean_statcast.csv', index=False) 
# after cleaning up the code, runtime went from over 10 mins to just over 3 mins
# code is still fully functional if you clean cell[4] is changed or cleaned in excel to match "bat_id_clean.csv"


In [18]:
import pandas as pd
new_df = pd.read_csv('all_clean_statcast.csv', low_memory=False)
# creating a pitcher index 
pitchers_team = new_df.loc[:,'pitch_team']
pitchers_name = new_df.loc[:,'player_name']
pitchers_index = new_df.loc[:,'pitcher']
frames_2 = [pitchers_team,pitchers_name,pitchers_index]
pitch_df = pd.DataFrame(frames_2)
pitch_df = pitch_df.transpose()
pitch_df = pitch_df.drop_duplicates(subset="pitcher")
# there's a few batters that had to pitch extra inning games, I wanna take them out to analyze if national pitchers get a big advantage from being able to pitch against pitchers all season
# batters in pitch_df [danny valencia, phillip ervin, mark reynolds, anthony rizzo, drew butera, carlos gomez, Enrique Hernandez \
# carlos tocci, ryan lamarre, alex blandino, ryan rua, jesus sucre, brandon guyer, chase d'arnaud, mitch garver, greg garcia\
# daniel robertson, ian happ, pablo sandoval, charlie culberson, Kendrys Morales, johnny field, cliff pennington, jd davis,
# jace peterson, Jake Smolinski, Jedd Gyorko, Taylor Motter, Jesmuel Valentin, Willians Astudillo, Jose Reyes, Jeff Mathis,
# Brandon Dixon, Scott Kingery, Tommy La Stella, Roman Quinn, Alex Avila, Cory Spangenberg, Bryan Holaday, Victor Caratini,
# Pedro Florimon, Chris Gimenez, Matt Davidson, Erik Kratz, Andrew Romine, Francisco Arcia, Hernan Perez, Daniel Descalso]
# Leaving Ohtani in the pitcher_id though he wont pitch in 2019. He's a legitimate hybrid pitcher and hitter
# Matt Davidson was an interesting find as well, he sports a 89.9 mph Fastball with a Changeup and Curveball
# i personally checked fangraphs every player who logged in less than 9 innings to see what type of player they were listed as
#pitch_df = pitch_df.drop([465840, 637131, 440133, 211694, 728599, 610084, 268171, 581720, 799568, 640819, \
                          #581712, 610072, 330077, 292458, 781849, 187706, 627492, 212460, 308963, 122824, 106211,\
                          #622017, 654759, 67289, 456473, 44844, 190077, 368773, 531463, 785508, 409466, 241708, 633668,\
                          #521540, 212486, 521607, 241258, 501660, 372544, 211696, 535755, 216380, 812482, 153000, 348357,\
                          #1428, 153013, 241271])

In [19]:
#
#new_df.head()
import pandas as pd
new_df = pd.read_csv('all_clean_statcast.csv', low_memory=False)
pd.set_option('display.max_columns', 50)
new_df.head()
new_df['player_name'].nunique()

799

In [20]:
len(pitch_df)

799

In [21]:
pitch_df.to_csv(f'pitcher_id.csv', index=False) 